# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../project.env")
sys.path.append(os.environ["PYTHONPATH"])

import pickle

In [2]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAINheader as train



Chosen class grouping: two-classes


Directory /Users/diego/Desktop/iteso/TOG/ exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/src exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/media exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/scores exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH1/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH2/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/two-classes exists. Continuing with execution
device: mps

In [3]:
TRAIN_classes = 'two-classes'

# Model training

## KNN

### Active Hand prediction

In [4]:
train.arch.find_best(sup.DATA_AH_PF, sup.active_hand_col, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.8892215568862275
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9021956087824351
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 3}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.907185628742515
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 5}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.908183632734531
	{'PH2': True, 

### Sign prediction

In [5]:
train.arch.find_best(sup.DATA_S_PF, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.54296875
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.55859375
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 2}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.560546875
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 3}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.572265625
	{'PH2': True, 'PH3': True, 'reducer': 'pca

In [6]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.6511627906976745
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.6744186046511628
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 4}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.6976744186046512
	{'PH2': True, 'PH3': True, 'reducer': 'kpca', 'kernel': 'poly', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 11}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.7209302325581395
	{'PH2'

## BERT

In [7]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_BERT_CODE)

updating best... 0.5116279069767442
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'two-classes', 'batch_size': 1024, 'input_dim': 1}
	{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'prajjwal1/bert-tiny', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 2e-05, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 2}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/BERT/Spv/prajjwal1/bert-tiny exists. Continuing with execution
updating best... 0.6511627906976745
	{'PH2': True, 'PH3': True, 'reducer': 'kpca', 'kernel': 'sigmoid', 'n': 2, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'two-classes', 'batch_size': 1024, 'input_dim': 2}
	{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'prajjwal1/bert-tiny', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 2e-05, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'

# Score saving

In [8]:
import pandas as pd
from datetime import datetime

## KNN

In [9]:
knn_scores_df = pd.DataFrame(sup.knn_score_tracker, columns=sup.knn_scores_columns)

In [10]:
knn_scores_df

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,k
0,two-classes,0.889222,AHpf,True,True,pca,None,1,1
1,two-classes,0.885230,AHpf,True,True,pca,None,1,2
2,two-classes,0.902196,AHpf,True,True,pca,None,1,3
3,two-classes,0.902196,AHpf,True,True,pca,None,1,4
4,two-classes,0.907186,AHpf,True,True,pca,None,1,5
...,...,...,...,...,...,...,...,...,...
7993,two-classes,0.767442,Spv,False,False,None,None,-1,27
7994,two-classes,0.790698,Spv,False,False,None,None,-1,28
7995,two-classes,0.744186,Spv,False,False,None,None,-1,29
7996,two-classes,0.790698,Spv,False,False,None,None,-1,30


In [11]:
knn_scores_df.sort_values(by="accuracy", ascending=False).head(50)

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,k
5301,two-classes,0.988281,Spf,False,False,None,None,-1,1
4712,two-classes,0.986328,Spf,False,True,umap,None,4,1
1303,two-classes,0.985030,AHpf,True,False,None,None,-1,2
562,two-classes,0.985030,AHpf,True,True,pca,None,4,5
1055,two-classes,0.984032,AHpf,True,True,kpca,cosine,6,2
500,two-classes,0.984032,AHpf,True,True,kpca,cosine,3,5
869,two-classes,0.983034,AHpf,True,True,kpca,cosine,5,2
499,two-classes,0.983034,AHpf,True,True,kpca,cosine,3,4
688,two-classes,0.983034,AHpf,True,True,kpca,cosine,4,7
684,two-classes,0.983034,AHpf,True,True,kpca,cosine,4,3


In [12]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_KNN_CODE))
knn_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_KNN_CODE, f"{now}.csv"), index=False)

Directory /Users/diego/Desktop/iteso/TOG/scores/two-classes/KNN exists. Continuing with execution


## BERT

In [13]:
bert_scores_df = pd.DataFrame(sup.bert_score_tracker, columns=sup.bert_scores_columns)

In [14]:
bert_scores_df

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,loadble,lr,optimizer,loss_fn,num_epochs
0,two-classes,0.511628,Spv,True,True,pca,None,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
1,two-classes,0.511628,Spv,True,True,kpca,poly,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
2,two-classes,0.511628,Spv,True,True,kpca,rbf,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
3,two-classes,0.511628,Spv,True,True,kpca,sigmoid,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
4,two-classes,0.511628,Spv,True,True,kpca,cosine,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,two-classes,0.511628,Spv,False,True,kpca,rbf,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
82,two-classes,0.000000,Spv,False,True,kpca,sigmoid,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
83,two-classes,0.000000,Spv,False,True,kpca,cosine,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
84,two-classes,0.000000,Spv,False,True,umap,None,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2


In [15]:
bert_scores_df.sort_values(by="accuracy", ascending=False).head(50)

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,loadble,lr,optimizer,loss_fn,num_epochs
49,two-classes,0.744186,Spv,False,True,pca,None,2,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
15,two-classes,0.674419,Spv,True,True,kpca,sigmoid,3,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
9,two-classes,0.651163,Spv,True,True,kpca,sigmoid,2,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
62,two-classes,0.558140,Spv,False,True,kpca,poly,4,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
17,two-classes,0.534884,Spv,True,True,umap,None,3,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
10,two-classes,0.534884,Spv,True,True,kpca,cosine,2,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
0,two-classes,0.511628,Spv,True,True,pca,None,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
1,two-classes,0.511628,Spv,True,True,kpca,poly,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
44,two-classes,0.511628,Spv,False,True,kpca,poly,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
45,two-classes,0.511628,Spv,False,True,kpca,rbf,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2


In [16]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_BERT_CODE))
bert_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_BERT_CODE, f"{now}.csv"), index=False)

Directory /Users/diego/Desktop/iteso/TOG/scores/two-classes/BERT exists. Continuing with execution
